### Read In Packages

In [1]:
from pathlib import Path
import os
import sys
from ultralytics import YOLO
import ultralytics
import IPython
import cv2
import torch
import torchvision
from typing import Generator

import matplotlib.pyplot as plt
import numpy as np

### Set Path

In [2]:
HOME = Path(os.getcwd())
HOME_STR = str(HOME)
print(HOME)

/Users/nelsonfarrell/Documents/Northeastern/5330/final_project


### Import Helpers

In [3]:
module_path = HOME_STR + '/utils'
if module_path not in sys.path:
    sys.path.append(module_path)
    
from process_video import clip_video

### Clip 2 Minute Soccer Video 

In [4]:
clip_vid = True
raw_file = "/data_raw/test_vid.mp4"
cliped_file = "/data_raw/test_vid_short.mp4"

if clip_vid:
    path_to_orignal = HOME_STR + raw_file
    path_to_clip = HOME_STR + cliped_file
    clip_video(path_to_orignal, path_to_clip, 20, 120)

### Set Paths to Video Clip

In [5]:
SOURCE_VIDEO_PATH = f'{HOME}/data_raw/test_vid_short.mp4'
SOURCE_VIDEO_PATH_2 = f'{HOME}/data_raw/soccer_short.mp4'
SOURCE_VIDEO_PATH_3 = f'{HOME}/data_raw/madrid_vs_city_short.mp4'

print(SOURCE_VIDEO_PATH)

/Users/nelsonfarrell/Documents/Northeastern/5330/final_project/data_raw/test_vid_short.mp4


### Import Model

In [8]:
ultralytics.checks()

Ultralytics YOLOv8.1.42 🚀 Python-3.11.8 torch-2.2.0 CPU (Apple M1 Pro)
Setup complete ✅ (10 CPUs, 16.0 GB RAM, 342.9/460.4 GB disk)


In [9]:
MODEL = "yolov8x.pt"
model = YOLO(MODEL)
model.fuse()

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


### Execute Model on Entire Clip

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model=yolov8x.pt source={SOURCE_VIDEO_PATH}

### Go Through Clip Frame by Frame

In [ ]:
def get_frame_genator(video_path: str) -> Generator[np.ndarray, None, None]:
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        raise Exception(f'Could not find Video at: {video_path}')
    success, frame = video.read()
    while success:
        yield frame
        success, frame = video.read()
    video.release()

In [ ]:
class_name_dict = model.model.names
print(class_name_dict)

In [6]:
person = 0
ball_class_id = 32

In [36]:
# open the video
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)

# check tha it was opened successfully
if not cap.isOpened():
    print("Error opening video")

# get frame
ret, frame = cap.read()

results = model(frame)
print(results)

while ret:
    results = model(frame)
    boxes = results[0].boxes.xyxy.numpy()
    classes = results[0].boxes.cls.numpy().astype(int)

    #if ball_class_id not in classes:
        

    for box, id in zip(boxes, classes):

        # draw bounding box
        start_point = (int(box[0]), int(box[1]))
        end_point = (int(box[2]), int(box[3]))
        color = (0, 0, 255) 
        thickness = 1
        cv2.rectangle(frame, start_point, end_point, color, thickness)

        # put text
        text = f'Player {id}'
        font = cv2.FONT_HERSHEY_COMPLEX
        font_scale = 0.5
        start_pt = (start_point[0] + 10, start_point[1] + 10)
        color = (0, 255, 255)
        thickness = 1
        type = cv2.LINE_AA
        cv2.putText(frame, text, start_pt, font, font_scale, color, thickness, type)
    ret, frame = cap.read()
cap.release()
print(frame)



0: 384x640 22 persons, 1 sports ball, 257.1ms
Speed: 1.7ms preprocess, 257.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwic

In [37]:
# open the video
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)

# check tha it was opened successfully
if not cap.isOpened():
    print("Error opening video")

# get frame
ret, frame = cap.read()

results = model(frame)


0: 384x640 22 persons, 1 sports ball, 238.3ms
Speed: 1.4ms preprocess, 238.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


In [42]:
classes = results[0].boxes.cls.numpy()
ball_idx = np.argwhere(classes == 32)
print(ball_idx[0][0])

1


In [52]:
bounding_boxes = results[0].boxes.xyxy.numpy()
ball_bounding_box = bounding_boxes[ball_idx]
ball_bounding_box = ball_bounding_box[0][0]
ball_bounding_box

array([     424.58,      533.13,      435.43,         544], dtype=float32)

In [53]:
x = int(ball_bounding_box[0])
y = int(ball_bounding_box[1])
w = int(ball_bounding_box[2]) - int(ball_bounding_box[0])
h = int(ball_bounding_box[3] - ball_bounding_box[1])

In [10]:
# Initialize Kalman Filter
kalman = cv2.KalmanFilter(4, 2)  # 4 dimensions (x, y, dx, dy), 2 measurements (x, y)

# Initialize state variables: [x, y, dx, dy]
kalman.statePre = np.array([0, 0, 0, 0], dtype=np.float32)

# Define transition matrix: [1 0 dt 0, 0 1 0 dt, 0 0 1 0, 0 0 0 1]
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                    [0, 1, 0, 1],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], dtype=np.float32)

# Define measurement matrix: [1 0 0 0, 0 1 0 0]
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], dtype=np.float32)

# Initialize video capture
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
frames_without_ball = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    classes = results[0].boxes.cls.numpy()
    if ball_class_id in classes:
        ball_idx = np.argwhere(classes == ball_class_id)

        bounding_boxes = results[0].boxes.xyxy.numpy()
        ball_bounding_box = bounding_boxes[ball_idx]
        ball_bounding_box = ball_bounding_box[0][0]

        x = int(ball_bounding_box[0])
        y = int(ball_bounding_box[1])
        w = int(ball_bounding_box[2]) - int(ball_bounding_box[0])
        h = int(ball_bounding_box[3] - ball_bounding_box[1])

        measurement = np.array([[x + w / 2], [y + h / 2]], dtype = np.float32)
        prediction = kalman.predict()
        kalman.correct(measurement)

        cv2.rectangle(frame, (int(x - w / 2), int(y - h / 2)),
            (int(x + w / 2), int(y + h / 2)), (0, 0, 255), 2)

    # Kalman prediction
    prediction = kalman.predict()

    # Kalman update
    kalman.correct(measurement)

    # Draw predicted position on frame
    predicted_x, predicted_y = prediction[:2]
    cv2.rectangle(frame, (int(predicted_x - w / 2), int(predicted_y - h / 2)),
                  (int(predicted_x + w / 2), int(predicted_y + h / 2)), (0, 255, 0), 2)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 22 persons, 1 sports ball, 252.6ms
Speed: 4.2ms preprocess, 252.6ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 241.0ms
Speed: 1.2ms preprocess, 241.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 1 sports ball, 231.3ms
Speed: 1.4ms preprocess, 231.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 1 sports ball, 228.0ms
Speed: 1.0ms preprocess, 228.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 228.3ms
Speed: 1.5ms preprocess, 228.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 1 sports ball, 225.0ms
Speed: 1.3ms preprocess, 225.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 1 sports ball, 228.3ms
Speed: 1.2ms preprocess, 228.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 

### Display Model Results

In [ ]:
print(results)

### Bounding Boxes

In [ ]:
results.boxes.xyxy

### Predicted Classes (Labels)

In [ ]:
results.boxes.cls

### Probabilities (Confidence)

In [ ]:
results.boxes.conf

### Display Frame

In [ ]:
SAVE_FRAME_FOLDER = HOME_STR+ "/processed_video_frames/"

if frame.ndim == 2:
    plt.figure(figsize = (10, 10))
    plt.imshow(frame, cmap = "gray")
else:
    plt.figure(figsize = (10, 10))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.savefig(SAVE_FRAME_FOLDER + "frame_.png")
    plt.close()


### Convert Tensor to Numpy

In [1]:
boxes = results.boxes.xyxy.numpy()
classes = results.boxes.cls.numpy().astype(int)


NameError: name 'results' is not defined

### Draw Boxes and Put Text

In [ ]:
for box, id in zip(boxes, classes):

    # draw bounding box
    start_point = (int(box[0]), int(box[1]))
    end_point = (int(box[2]), int(box[3]))
    color = (0, 0, 255) 
    thickness = 1
    cv2.rectangle(frame, start_point, end_point, color, thickness)

    # put text
    text = f'Player {id}'
    font = cv2.FONT_HERSHEY_COMPLEX
    font_scale = 0.5
    start_pt = (start_point[0] + 10, start_point[1] + 10)
    color = (0, 255, 255)
    thickness = 1
    type = cv2.LINE_AA
    cv2.putText(frame, text, start_pt, font, font_scale, color, thickness, type)

In [ ]:
if frame.ndim == 2:
    plt.figure(figsize = (10, 10))
    plt.imshow(frame, cmap = "gray")
else:
    plt.figure(figsize = (10, 10))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.savefig(SAVE_FRAME_FOLDER + "frame_1.png")
    plt.close()